<a href="https://colab.research.google.com/github/Codingturtle108/GraphTheoryWithPython/blob/main/BagofWordsFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [214]:
Stop_words = stopwords.words('english')
def preprocess_text(text):
  text = re.sub(r'[^a-zA-z0-9\s]',' ',text)
  text = [i.lower() for i in text.split() if i not in Stop_words]
  return ' '.join(text)
Classifier = lambda word: True if word=='spam' else False

In [215]:
df = pd.read_csv('spam.csv',encoding='latin1')
df=df[['v1','v2']]
df.rename(columns={'v1':'Spam','v2':'Text'},inplace=True)
df['Spam']=df['Spam'].apply(Classifier)



In [216]:
df['Text']=df['Text'].apply(preprocess_text)
df=df.sample(frac=1)

In [217]:
df

,Spam,Text
703,False,thats bit weird even supposed happening but go...
3870,False,yeah usual guy town definitely people around i...
1945,False,can _ call 10 10 make sure dat woken
4515,True,congrats 2 mobile 3g videophones r call 090617...
1886,True,dear 0776xxxxxxx u invited xchat this final at...
...,...,...
3149,False,yo howz u girls never rang india l
844,False,k k going
2423,False,lmao fun
5039,True,natalie 20 f inviting friend reply yes 165 no ...


In [218]:
spam_df = df.loc[df['Spam']==True]
normie_df = df.loc[df['Spam']==False]
len(spam_df),len(normie_df)
Pspam = len(spam_df)/(len(spam_df)+len(normie_df))
Pnormie = len(normie_df)/(len(spam_df)+len(normie_df))

In [219]:
spam_df.head()

,Spam,Text
4515,True,congrats 2 mobile 3g videophones r call 090617...
1886,True,dear 0776xxxxxxx u invited xchat this final at...
3989,True,bank granite issues strong buy explosive pick ...
4010,True,please call customer service representative fr...
3358,True,sorry i missed call let talk time i 07090201529


In [220]:
train_df = df.iloc[:int(len(df)*0.7)]
test_df  = df.iloc[int(len(df)*0.7):]

In [221]:
train_spam_words = ' '.join(train_df[train_df.Spam==True].Text).split(' ')
train_normie_words = ' '.join(train_df[train_df.Spam==False].Text).split(' ')

In [222]:
len(train_spam_words),len(train_normie_words)

(10105, 31402)

In [223]:
from collections import Counter
bow_trn_spam = Counter(train_spam_words)
bow_trn_normie = Counter(train_normie_words)
spam_probabilites = { key:value/len(train_spam_words) for key, value in bow_trn_spam.items()}
normie_probabilites={ key:value/len(train_normie_words) for key, value in bow_trn_normie.items()}

In [224]:
def predict_spam(text):
  spam_score=np.log(Pspam)
  normie_score = np.log(Pnormie)
  for word in text:
    if word in train_spam_words:
      spam_score += np.log(spam_probabilites[word])
    if word in train_normie_words:
      normie_score += np.log(normie_probabilites[word])
  if spam_score > normie_score:
    return 1
  return 0



In [225]:
train_df['Prediction'] = train_df['Text'].apply(predict_spam)

<ipython-input-225-786b224a6c1b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Prediction'] = train_df['Text'].apply(predict_spam)


In [226]:
train_df

,Spam,Text,Prediction
703,False,thats bit weird even supposed happening but go...,1
3870,False,yeah usual guy town definitely people around i...,1
1945,False,can _ call 10 10 make sure dat woken,0
4515,True,congrats 2 mobile 3g videophones r call 090617...,0
1886,True,dear 0776xxxxxxx u invited xchat this final at...,1
...,...,...,...
565,False,ill call u 2mrw ninish address icky american f...,0
498,False,kate jackson rec center 7ish right,0
1588,False,at 4 let go bill millers,0
3247,False,also track lighters find,1


In [227]:
Correct=0
for i in range(len(train_df)):
  if train_df.iloc[i,0]==train_df.iloc[i,2]:
    Correct +=1
Correct/len(train_df)

0.3730769230769231